# 环境配置（此处使用AutoDL线上平台）
Cuda    12.1

GPU     RTX 4090D(24GB) * 1

CPU     15 vCPU Intel(R) Xeon(R) Platinum 8474C

Python            3.10(ubuntu22.04)

PyTorch           2.1.0

transformers      4.41.2

seqeval           1.2.2

pytorch-crf       0.7.2

In [208]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertModel, AdamW
from seqeval.metrics import classification_report # 可以查看这个教程，https://www.cnblogs.com/jclian91/p/12459688.html
from torchcrf import CRF # 这里使用 pytorch-crf 的实现 https://pytorch-crf.readthedocs.io/en/stable/

In [209]:
# 参数设置
BATCH_SIZE = 16  # 每个 batch 包含 N 个样本
EPOCHS = 3  # 训练 3 个 epoch
LEARNING_RATE = 5e-5  # 学习率

MAX_SEQ_LEN = 512 # 可处理的最大句子长度，一般取决于预训练模型


In [210]:
# 标签定义
LABELS = [
    "O", "B-NAME", "I-NAME", "B-CONT", "I-CONT", "B-EDU", "I-EDU", "B-TITLE", "I-TITLE", "B-ORG", "I-ORG", "B-RACE", "I-RACE", "B-PRO", "I-PRO", "B-LOC", "I-LOC", "NONE"
] # 这里我们加入了一个NONE标签

LABELS_NUM = len(LABELS) - 1 # 分类数量，也就是我们这里是一个几分类问题，NONE不是一个要训练要学习的分类，是一个特殊标记，所以要减掉1，感觉这个时限不太优雅，后期需要修改
NONE_LABEL_IDX = len(LABELS) - 1 #这里是个数字记录了，我们设置的NONE标签的下标，NONE标签是荣放在最后一位

label2id = {label: i for i, label in enumerate(LABELS)}  # 标签到 ID 的映射
id2label = {i: label for i, label in enumerate(LABELS)}  # ID 到标签的映射
print('标签种类：', LABELS_NUM)
print(label2id)
print(id2label)

标签种类： 17
{'O': 0, 'B-NAME': 1, 'I-NAME': 2, 'B-CONT': 3, 'I-CONT': 4, 'B-EDU': 5, 'I-EDU': 6, 'B-TITLE': 7, 'I-TITLE': 8, 'B-ORG': 9, 'I-ORG': 10, 'B-RACE': 11, 'I-RACE': 12, 'B-PRO': 13, 'I-PRO': 14, 'B-LOC': 15, 'I-LOC': 16, 'NONE': 17}
{0: 'O', 1: 'B-NAME', 2: 'I-NAME', 3: 'B-CONT', 4: 'I-CONT', 5: 'B-EDU', 6: 'I-EDU', 7: 'B-TITLE', 8: 'I-TITLE', 9: 'B-ORG', 10: 'I-ORG', 11: 'B-RACE', 12: 'I-RACE', 13: 'B-PRO', 14: 'I-PRO', 15: 'B-LOC', 16: 'I-LOC', 17: 'NONE'}


In [211]:
# 工具函数，读取数据，并按句子切分
def read_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()  # 读取文件中的所有行

    texts, labels = [], []
    text, label = [], []
    for line in lines:
        if line.strip() == "": # 句子切分处，这里检测是否是空行，我们的语料中使用空行切割句子，其实也可以替换为\n\n
            if text:
                texts.append(text)  # 将一个句子的文本添加到文本列表中
                # 将一个句子的标签转换为 ID ，添加到标签列表中
                labels.append([label2id[l] for l in label])
                text, label = [], []
        else: # 非句子切分处，也就是如同“项 B-TITLE”这种形式的，带有标注的行
            token, tag = line.strip().split()  # 分割文本和标签，split默认使用空格切分，例如“学 B-EDU”会切分为“学”“B-EDU”
            text.append(token)
            label.append(tag)

    if text:
        texts.append(text)
        labels.append([label2id[l] for l in label])

    return texts, labels  # 返回文本和标签列表，这两个都是二维数组
# 读取数据
train_texts, train_labels = read_data("train.txt")
test_texts, test_labels = read_data("test.txt")

print('训练集句子数量：', len(train_texts), train_texts[0])
print('训练集句子标签：', len(train_labels), train_labels[0])
print('测试集句子数量：', len(test_texts), test_texts[0])
print('测试集句子标签：', len(test_labels), test_labels[0])

训练集句子数量： 3 ['高', '勇', '：', '男', '，', '中', '国', '国', '籍', '，', '无', '境', '外', '居', '留', '权', '，']
训练集句子标签： 3 [1, 2, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0]
测试集句子数量： 2 ['常', '建', '良', '，', '男', '，']
测试集句子标签： 2 [1, 2, 2, 0, 0, 0]


In [212]:
# 管理数据
class NERDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)  # 返回数据集大小

    def __getitem__(self, idx):
        # 这里就是按一个句子，一个句子去处理的
        text = self.texts[idx]  # 获取第 idx 个样本（句子）的文本，① 这是一个句子的文本
        labels = self.labels[idx]  # 获取第 idx 个样本（句子）的标签，这是一个句子的BIO标签，② 但是使用数字形式标注

        # 转换成bert的token格式
        encoding = self.tokenizer(
            text,
            is_split_into_words=True,
            truncation=True,
            padding='max_length',
            max_length=MAX_SEQ_LEN,
            return_tensors="pt"
        )
        # print(111, text)
        # 111 ['高', '勇', '：', '男', '，', '中', '国', '国', '籍', '，', '无', '境', '外', '居', '留', '权', '，']
        
        # print(222, labels)
        # 222 [1, 2, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0]
        
        # print(333, encoding)
        # encoding该对象有3个属性，但后面会被补充为4个属性
        # 这3个属性分别是
        # input_ids：注意是二维数组，101是<cls>特殊标记，然后不足512，padding到512
        # input_ids：tensor([[ 101, 7770, 1235, 8038, 4511, 8024,  704, 1744, 1744, 5093, 8024, 3187, 1862, 1912, 2233, 4522, 3326, 8024,  102,    0,    0,    0,    0,    0...]])
        
        # token_type_ids：用于标记两个句子，比如一个问题标记为0，答案标记为1，我们这里都标记为0
        # token_type_ids：tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...]])

        # attention_mask：设置是否参与注意力机制，注意cls位置标记为了1，但pad标记为0
        # attention_mask：tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0...

        
        word_ids = encoding.word_ids()  # 获取每个 token 所对应的单词索引
        label_ids = []
        # 关于word_ids
        # print(444, word_ids)
        # 也是512位，[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, None, None, None, None, None, None, None, ....]
        # 作用：如果某个子词是原始句子中的一个token，它将返回该单词在原始文本中的索引。
        # 而如果子词是由于tokenization过程产生的，其无法直接映射到原始单词（比如因为标点符号、特殊字符或单词被切分产生的子词），它会返回None。其实这个在处理英文介词时更为常用
        # word_ids列表的长度与input_ids相同，意味着每个tokenized的子词都有一个对应的word_id。

        # 下面主要用于生成第四种数据，更改后的label数据（前面的三种数据分别是：text、labels、encoding），主要是将原来的label拓展到512维
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(0)  # 设置word_ids = None的位置填充填充的标签，仅仅是一个标记，不要和正式的标签id重合。但是搭配CRF使用时，我暂且把这里设置维0，否则运算会报错 ⚠️ 这主要是因为这个代码从bert-Softmax修改而来，那个模型中使用-100作为标记，其实实现bert-bilstm-crf时可以尝试换一个方案
            else:
                label_ids.append(labels[word_idx])
            # 下面的代码有些冗余
            # elif word_idx != previous_word_idx:
            #     label_ids.append(labels[word_idx])  # 第一次出现的单词的标签
            # else:
            #     label_ids.append(
            #         labels[word_idx] if word_idx is not None else -100)
            # previous_word_idx = word_idx
        encoding['labels'] = torch.tensor(label_ids)  # 将标签添加到encoding中，这下encoding就有4个属性了：input_ids、token_type_ids、attention_mask、labels
        # print(444, encoding['labels'])
        # None的位置都被标记为了17
        # 444 tensor(tensor([17,  1,  2,  0,  0,  0,  3,  4,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0, 17, 17, 17, 17, 17, 17, 17, 17, 
       
        # 返回encoding
        # 理解下squeeze：torch.tensor([[1,2,3]]).squeeze(0)，返回tensor([1, 2, 3])，从二维压缩到了一维
        # Transformers库的tokenizer，用于存储文本处理后的各种信息，如input_ids、attention_mask等。这些张量可能具有一个额外的维度，通常是批处理维度（即使只有一个样本，也会默认添加一个维度，形状为(1, ...))。
        # squeeze(0)：这是PyTorch中的一个操作，用于移除张量中大小为1的维度。在这里，0指定要移除第一个维度。如果val是一个形状为(1, ...,)的张量，val.squeeze(0)就会返回一个形状为(...)的张量，即去掉了那个单元素的批次维度，使得处理单个样本时更加方便。
        # 返回的就是一个对象{input_ids, token_type_ids, attention_mask, labels}, 每个属性对应的都是一个512维向量
        return {key: val.squeeze(0) for key, val in encoding.items()}


In [213]:
# 创建模型
class BertForNER(nn.Module):
    def __init__(self, num_labels):
        super(BertForNER, self).__init__()
        
        # BERT
        self.bert = BertModel.from_pretrained("bert-base-chinese")

        # LSTM
        self.lstm_hiden = 128 # 隐层神经元个数
        self.bilstm = nn.LSTM(
            768, # 输入维度
            self.lstm_hiden, # 隐层神经元个数
            1, # LSTM层数
            bidirectional=True, # 双向LSTM
            batch_first=True, # batch_first: 如果为 True，则输入和输出的形状为 (batch_size, seq_len, feature_dim)，否则为 (seq_len, batch_size, feature_dim)。
            dropout=0.1 # 随机失活
        )

        # LINEAR，也有说法说，这个线性层不是必要的，需要仔细研究一下CRF的输入
        self.linear = nn.Linear(self.lstm_hiden * 2, LABELS_NUM) # 将LSTM的输出降维到标签个数，用于之后分类

        # CRF
        self.crf = CRF(LABELS_NUM, batch_first=True) # 创建模型：model = CRF(num_tags)  https://pytorch-crf.readthedocs.io/en/stable/

    def forward(self, input_ids, attention_mask=None, labels=None):
        # BERT
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # seq_out = bert_output[0]  # [batchsize, max_len, 768]    # 尝试一下取last_hidden_state
        seq_out = bert_output.last_hidden_state  # [batchsize, max_len, 768]    # 尝试一下取last_hidden_state
        batch_size = seq_out.size(0)

        # LSTM
        seq_out, _ = self.bilstm(seq_out)
        seq_out = seq_out.contiguous().view(-1, self.lstm_hiden * 2)
        seq_out = seq_out.contiguous().view(batch_size, MAX_SEQ_LEN, -1)
        
        # LINEAR
        seq_out = self.linear(seq_out)
        # print(111, seq_out.shape, seq_out)
        # 下面的维度“2”，代表两个句子，这个取决于我们设置的batch_size
        # 512是每个句子包含512个单词，如果不够或者过长的话，会截长补短
        # 17是每个单词进行17分类，这里存储的就是每个单词分配给每个类的概率
        # 111 torch.Size([2, 512, 17]) tensor([[[ 0.1956, -0.0638,  0.1322,  ..., -0.0259, -0.0147, -0.0049],
        #  [ 0.1993,  0.2236,  0.2785,  ..., -0.0203, -0.0703,  0.1671],
        #  [ 0.1984,  0.2107,  0.0458,  ...,  0.0763, -0.0182,  0.1876],

        # CRF
        # crf的输出结果是一个二维数组
        # 其中是每个数组对应一个句子
        # 数组中的元素就是这个句子当中的单词最可能得分类
        # 注意再考虑句子中的单词时，只考虑Attention mask = 1的，不过在我们这里<cls>也被标记为1
        # 222 [[2, 2, 0, 10, 1, 16, 0, 15, 15, 1, 13, 10, 13, 16, 13, 10, 1, 13, 2, 2], [14, 1, 13, 10, 0, 10, 1, 10, 1, 14, 1, 4, 4, 10, 1, 4, 4, 10, 1]]
        logits = self.crf.decode(seq_out, mask=attention_mask.bool()) # 这个mask用于忽略掉填充进来的，比如pad之类的信息
        # print(222, logits)

        loss = None
        if labels is not None: # 如果传入的有标签，此时我们应该是在进行训练，使用的是带标签数据，而不是在进行预测
            loss = -self.crf(seq_out, labels,
                             mask=attention_mask.bool(), reduction='mean')
        return loss, logits  # 返回损失和分类结果

In [214]:
# 加载预训练的BERT模型和分词器
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

In [215]:
# 测试用的数据，仅用于查看输出
# NERDataset(train_texts, train_labels, tokenizer)[0]

In [216]:
# 数据加载
train_dataset = NERDataset(train_texts, train_labels, tokenizer)
test_dataset = NERDataset(test_texts, test_labels, tokenizer)

print('训练集句子数量：', len(train_dataset))
print('训练集第一个句子：', train_dataset[0])

训练集句子数量： 3
训练集第一个句子： {'input_ids': tensor([ 101, 7770, 1235, 8038, 4511, 8024,  704, 1744, 1744, 5093, 8024, 3187,
        1862, 1912, 2233, 4522, 3326, 8024,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
     

In [217]:
# 创建模型
model = BertForNER(num_labels=LABELS_NUM)  # 初始化模型

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
d:\softwares\python\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero 

In [218]:
# 切分数据，使用dataloader
train_dataloader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True)  # 训练数据加载器
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)  # 测试数据加载器

In [219]:
# 优化器
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)  # AdamW 优化器

d:\softwares\python\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [220]:
# 训练函数
def train(device):
    model.train()  # 设置模型为训练模式
    model.to(device)  # 将模型移动到指定设备
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch_idx, batch in enumerate(train_dataloader):
            optimizer.zero_grad()  # 清空梯度
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            # 注意这里每一个batch，因为我们设置batch-size=2，所以这里包含两个句子，每个句子的input_ids, labels等属性都是512维
            # print(111, batch['input_ids'].shape, batch['input_ids'])
            # print(222, batch['token_type_ids'].shape, batch['token_type_ids'])
            # print(333, batch['attention_mask'].shape, batch['attention_mask'])
            # print(444, batch['labels'].shape, batch['labels'])
            # 111 torch.Size([2, 512]) tensor([[101, 123, 121,  ...,   0,   0,   0],
            #         [101, 122, 130,  ...,   0,   0,   0]])
            # 222 torch.Size([2, 512]) tensor([[0, 0, 0,  ..., 0, 0, 0],
            #         [0, 0, 0,  ..., 0, 0, 0]])
            # 333 torch.Size([2, 512]) tensor([[1, 1, 1,  ..., 0, 0, 0],
            #         [1, 1, 1,  ..., 0, 0, 0]])
            # 444 torch.Size([2, 512]) tensor([[17,  0,  0,  ..., 17, 17, 17],
            #         [17,  0,  0,  ..., 17, 17, 17]])
            loss, _ = model(
                input_ids, attention_mask=attention_mask, labels=labels)
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
            total_loss += loss.item()
            if (batch_idx + 1) % 1 == 0:  # 每 10 个 batch 打印一次进度
                print(
                    f"Epoch {epoch + 1}/{EPOCHS}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item()}")
        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{EPOCHS}, Average Loss: {avg_loss}")

In [221]:

# 评估函数
def evaluate(device):
    model.eval()  # 设置模型为评估模式
    model.to(device)  # 将模型移动到指定设备
    # 评估也是分batch运行的
    # 只是后面生成评估报告，真正计算f1时，需要综合各个batch的结果
    # 综合各个batch的内容就放在下面两个数组中
    true_labels = []
    pred_labels = []
    for batch_idx, batch in enumerate(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        with torch.no_grad():
            _, logits = model(input_ids, attention_mask=attention_mask)

        # 下面是获取真值标签和预测标签
        # 并且获取到到的都是id形式，需要转换回原来的标签形式
        true_labels_batch = []
        pred_labels_batch = []
        for i in range(len(logits)):
            true_labels_batch_ids = labels[i][:len(logits[i])].tolist() # 截断label，因为我们这里的label，按前面的逻辑是512位，但是crf输出的logits，按前面的逻辑只包含Attention_mask为1的部分
            true_labels_batch.append([id2label[id] for id in true_labels_batch_ids])
            pred_labels_batch.append([id2label[id] for id in logits[i]])
        # print(111, len(true_labels_batch))
        # print(222, len(pred_labels_batch))
  
        true_labels.extend(true_labels_batch)
        pred_labels.extend(pred_labels_batch)

        # 只考虑有效的标签
        # true_labels_batch = [[id2label[l] for l in label if l != 17]
        #                      for label in labels.cpu().numpy()]
        # pred_labels_batch = [[id2label[p] for p, l in zip(
        #     pred, label) if l != 17] for pred, label in zip(predictions.cpu().numpy(), labels.cpu().numpy())]
        # # print(111, true_labels_batch)
        # # print(222, pred_labels_batch)
        # true_labels.extend(true_labels_batch)
        # pred_labels.extend(pred_labels_batch)

        if (batch_idx + 1) % 10 == 0:  # 每 10 个 batch 打印一次进度
            print(f"Evaluating Batch {batch_idx + 1}/{len(test_dataloader)}")

    # 使用 seqeval 计算评价指标
    print('参与评估的句子数量：', len(true_labels), len(pred_labels))
    print("Evaluation Results:")
    # true_labels、pred_labels是格式、大小一样的两个数组，都是二维数组，一个是真值，一个是预测值
    # [['O', 'B-NAME', 'I-NAME', 'I-NAME', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PRO', 'I-PRO', 'B-EDU', 'I-EDU', 'O', 'B-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'O']]
    # [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
    print(true_labels)
    print(pred_labels)
    print(classification_report(true_labels, pred_labels))

In [222]:
# 预测函数
def predict(text, device):
    model.eval()  # 设置模型为评估模式
    model.to(device)  # 将模型移动到指定设备
    tokens = tokenizer.tokenize(text)  # 对输入文本进行分词，这里是先拆分成单个汉字，单个汉字的形式，和我们的train语料保持一致
    # print(111, text)
    # print(222, tokens)
    encoding = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        padding='max_length',
        max_length=512
    )
    # print(333, encoding)

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        _, logits = model(input_ids, attention_mask=attention_mask)
    predictions = torch.tensor(logits)
    # print(444, predictions[0])

    predicted_labels = [id2label[p] for p in predictions[0].tolist()]
    # print(555, predicted_labels)
    return list(zip(tokens, predicted_labels[1:len(tokens) + 1])) # 这个1是去除cls位置的预测

In [223]:
# 训练
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(device)

Epoch 1/3, Batch 1/2, Loss: 127.58329010009766
Epoch 1/3, Batch 2/2, Loss: 52.27286911010742
Epoch 1/3, Average Loss: 89.92807960510254
Epoch 2/3, Batch 1/2, Loss: 46.042816162109375
Epoch 2/3, Batch 2/2, Loss: 158.37928771972656
Epoch 2/3, Average Loss: 102.21105194091797
Epoch 3/3, Batch 1/2, Loss: 92.01494598388672
Epoch 3/3, Batch 2/2, Loss: 37.671669006347656
Epoch 3/3, Average Loss: 64.84330749511719


In [224]:
# 评估
evaluate(device)

参与评估的句子数量： 2 2
Evaluation Results:
[['O', 'B-NAME', 'I-NAME', 'I-NAME', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PRO', 'I-PRO', 'B-EDU', 'I-EDU', 'O', 'B-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'O']]
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
              precision    recall  f1-score   support

         EDU       0.00      0.00      0.00         1
        NAME       0.00      0.00      0.00         1
         PRO       0.00      0.00      0.00         1
       TITLE       0.00      0.00      0.00         1

   micro avg       0.00      0.00      0.00         4
   macro avg       0.00      0.00      0.00         4
weighted avg       0.00      0.00      0.00         4



d:\softwares\python\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\softwares\python\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [225]:
# 预测
sample_text = "韦棣华女士曾任教于文华书院"
prediction = predict(sample_text, device)
print(prediction)

[('韦', 'O'), ('棣', 'O'), ('华', 'O'), ('女', 'O'), ('士', 'O'), ('曾', 'O'), ('任', 'O'), ('教', 'O'), ('于', 'O'), ('文', 'O'), ('华', 'O'), ('书', 'O'), ('院', 'O')]
